# EDB: fully parametrized CPWG design

This example shows how you can use HFSS 3D Layout to create a parametric design
for a CPWG (coplanar waveguide with ground).

## Perform required imports

Perform required imports. Importing the ``Hfss3dlayout`` object initializes it
on version 2023 R2.

In [1]:
import os

In [2]:
import numpy as np
import pyaedt

In [3]:
from pyedb import Edb
from pyedb.generic.general_methods import (
    generate_unique_folder_name,
    generate_unique_name,
)

## Set non-graphical mode

Set non-graphical mode. The default is ``False``.

In [4]:
non_graphical = False

## Launch EDB

In [5]:
aedb_path = os.path.join(generate_unique_folder_name(), generate_unique_name("pcb") + ".aedb")
print(aedb_path)

# Select EDB version (change it manually if needed, e.g. "2024.1")
edb_version = "2024.1"
print(f"EDB version: {edb_version}")
aedt_version = edb_version

edbapp = Edb(edbpath=aedb_path, edbversion=edb_version)

C:\Users\ansys\AppData\Local\Temp\pyedb_prj_7PW\pcb_YS9LI9.aedb
EDB version: 2024.1


PyAEDT INFO: Logger is initialized in EDB.


PyAEDT INFO: legacy v0.19.dev0


PyAEDT INFO: Python version 3.10.11 (tags/v3.10.11:7d4cc5a, Apr  5 2023, 00:38:17) [MSC v.1929 64 bit (AMD64)]


PyAEDT INFO: EDB C:\Users\ansys\AppData\Local\Temp\pyedb_prj_7PW\pcb_YS9LI9.aedb created correctly.


PyAEDT INFO: EDB initialized.


## Define parameters

In [6]:
params = {
    "$ms_width": "0.4mm",
    "$ms_clearance": "0.3mm",
    "$ms_length": "20mm",
}
for par_name in params:
    edbapp.add_project_variable(par_name, params[par_name])

## Create s symmetric stackup

In [7]:
edbapp.stackup.create_symmetric_stackup(2)
edbapp.stackup.plot()

Material 'copper' does not exist in material library. Intempt to create it from syslib.


Material 'FR4_epoxy' does not exist in material library. Intempt to create it from syslib.


Material 'SolderMask' does not exist in material library. Intempt to create it from syslib.


C:\actions-runner\_work\pyedb\pyedb\.venv\lib\site-packages\pyedb\dotnet\edb_core\stackup.py:2889: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


<module 'matplotlib.pyplot' from 'C:\\actions-runner\\_work\\pyedb\\pyedb\\.venv\\lib\\site-packages\\matplotlib\\pyplot.py'>

## Draw planes

In [8]:
plane_lw_pt = ["0mm", "-3mm"]
plane_up_pt = ["$ms_length", "3mm"]

top_layer_obj = edbapp.modeler.create_rectangle(
    "TOP", net_name="gnd", lower_left_point=plane_lw_pt, upper_right_point=plane_up_pt
)
bot_layer_obj = edbapp.modeler.create_rectangle(
    "BOTTOM", net_name="gnd", lower_left_point=plane_lw_pt, upper_right_point=plane_up_pt
)
layer_dict = {"TOP": top_layer_obj, "BOTTOM": bot_layer_obj}

## Draw a trace

In [9]:
trace_path = [["0", "0"], ["$ms_length", "0"]]
edbapp.modeler.create_trace(
    trace_path,
    layer_name="TOP",
    width="$ms_width",
    net_name="sig",
    start_cap_style="Flat",
    end_cap_style="Flat",
)

## Create a trace to plane clearance

In [10]:
poly_void = edbapp.modeler.create_trace(
    trace_path,
    layer_name="TOP",
    net_name="gnd",
    width="{}+2*{}".format("$ms_width", "$ms_clearance"),
    start_cap_style="Flat",
    end_cap_style="Flat",
)
edbapp.modeler.add_void(layer_dict["TOP"], poly_void)

True

## Create a ground via padstack and place ground stitching vias

In [11]:
edbapp.padstacks.create(
    padstackname="GVIA",
    holediam="0.3mm",
    paddiam="0.5mm",
)

yloc_u = "$ms_width/2+$ms_clearance+0.25mm"
yloc_l = "-$ms_width/2-$ms_clearance-0.25mm"

for i in np.arange(1, 20):
    edbapp.padstacks.place([str(i) + "mm", yloc_u], "GVIA", net_name="GND")
    edbapp.padstacks.place([str(i) + "mm", yloc_l], "GVIA", net_name="GND")

PyAEDT INFO: Padstack GVIA create correctly


## Save and close EDB

In [12]:
edbapp.save_edb()
edbapp.close_edb()

PyAEDT INFO: EDB file save time: 0.00ms


PyAEDT INFO: EDB file release time: 15.62ms


True

## Open EDB in AEDT

In [13]:
aedt_version = edb_version

In [14]:
h3d = pyaedt.Hfss3dLayout(
    projectname=aedb_path,
    specified_version=aedt_version,
    non_graphical=non_graphical,
    new_desktop_session=True,
)

PyAEDT WARNING: Argument `projectname` is deprecated for method `__init__`; use `project` instead.


PyAEDT WARNING: Argument `specified_version` is deprecated for method `__init__`; use `version` instead.


PyAEDT WARNING: Argument `new_desktop_session` is deprecated for method `__init__`; use `new_desktop` instead.


PyAEDT INFO: Initializing new Desktop session.


PyAEDT INFO: StdOut is enabled


...

PyAEDT INFO: Python version 3.10.11 (tags/v3.10.11:7d4cc5a, Apr  5 2023, 00:38:17) [MSC v.1929 64 bit (AMD64)]


PyAEDT INFO: AEDT 2024.1.0 Build Date 2023-11-27 22:16:18


PyAEDT INFO: EDB folder C:\Users\ansys\AppData\Local\Temp\pyedb_prj_7PW\pcb_YS9LI9.aedb has been imported to project pcb_YS9LI9


PyAEDT INFO: Active Design set to Cell_UPBGBG


PyAEDT INFO: Aedt Objects correctly read


## Create wave ports

In [15]:
h3d.create_edge_port("line_3", 0, iswave=True, wave_vertical_extension=10, wave_horizontal_extension=10)
h3d.create_edge_port("line_3", 2, iswave=True, wave_vertical_extension=10, wave_horizontal_extension=10)

PyAEDT WARNING: Argument `iswave` is deprecated for method `create_edge_port`; use `is_wave_port` instead.


PyAEDT INFO: Loading Modeler.


PyAEDT INFO: Modeler loaded.


PyAEDT INFO: EDB loaded.


PyAEDT INFO: Layers loaded.


...

PyAEDT ERROR:     is_wave_port = True 


PyAEDT ERROR:     wave_horizontal_extension = 10 


PyAEDT ERROR:     wave_vertical_extension = 10 


PyAEDT ERROR: **************************************************************


False

## Edit airbox extents

In [16]:
h3d.edit_hfss_extents(air_vertical_positive_padding="10mm", air_vertical_negative_padding="1mm")

True

## Create setup

In [17]:
setup = h3d.create_setup()
setup["MaxPasses"] = 2
setup["AdaptiveFrequency"] = "3GHz"
setup["SaveAdaptiveCurrents"] = True
h3d.create_linear_count_sweep(
    setupname=setup.name,
    unit="GHz",
    freqstart=0,
    freqstop=5,
    num_of_freq_points=1001,
    sweepname="sweep1",
    sweep_type="Interpolating",
    interpolation_tol_percent=1,
    interpolation_max_solutions=255,
    save_fields=False,
    use_q3d_for_dc=False,
)

PyAEDT WARNING: Argument `setupname` is deprecated for method `create_linear_count_sweep`; use `setup` instead.


PyAEDT WARNING: Argument `freqstart` is deprecated for method `create_linear_count_sweep`; use `start_frequency` instead.


PyAEDT WARNING: Argument `freqstop` is deprecated for method `create_linear_count_sweep`; use `stop_frequency` instead.


PyAEDT WARNING: Argument `sweepname` is deprecated for method `create_linear_count_sweep`; use `name` instead.


PyAEDT INFO: Linear count sweep sweep1 has been correctly created.


## Plot layout

In [18]:
h3d.modeler.edb.nets.plot(None, None, color_by_net=True)

cp_name = h3d.modeler.clip_plane()

PyAEDT INFO: Logger is initialized in EDB.


PyAEDT INFO: legacy v0.19.dev0


PyAEDT INFO: Python version 3.10.11 (tags/v3.10.11:7d4cc5a, Apr  5 2023, 00:38:17) [MSC v.1929 64 bit (AMD64)]


PyAEDT INFO: Database pcb_YS9LI9.aedb Opened in 2024.1


PyAEDT INFO: Cell Cell_UPBGBG Opened


PyAEDT INFO: Builder was initialized.


PyAEDT INFO: EDB initialized.


PyAEDT INFO: Nets Point Generation time 0.016 seconds


C:\actions-runner\_work\pyedb\pyedb\.venv\lib\site-packages\pyedb\generic\plot.py:147: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


## Solve the active design

Uncomment the following code to start the HFSS solver and perform post processing.

In [19]:
# h3d.analyze()

# solutions = h3d.get_touchstone_data()[0]
# solutions.log_x = False
# solutions.plot()

# h3d.post.create_fieldplot_cutplane(
#     cp_name, "Mag_E", h3d.nominal_adaptive, intrinsincDict={"Freq": "3GHz", "Phase": "0deg"}
# )

## Save AEDT

In [20]:
aedt_path = aedb_path.replace(".aedb", ".aedt")
h3d.logger.info("Your AEDT project is saved to {}".format(aedt_path))
h3d.save_project()

PyAEDT INFO: Your AEDT project is saved to C:\Users\ansys\AppData\Local\Temp\pyedb_prj_7PW\pcb_YS9LI9.aedt


PyAEDT INFO: Project pcb_YS9LI9 Saved correctly


True

## Release AEDT

In [21]:
h3d.release_desktop()

PyAEDT INFO: Desktop has been released and closed.


True